European Central Bank (ECB) API with Python
=====

## Data on Eurozone countries

-----

*September 3, 2017*<br>
*@bd_econ*

ECB API Documentation is [here](https://sdw-wsrest.ecb.europa.eu/web/generator/index.html)

This example requests the interest rate by country on 10-year bonds over the period since 2012.

In [1]:
import requests
import pandas as pd

## Parameters/Settings

List of data sets is [here](https://sdw-wsrest.ecb.europa.eu/service/dataflow)

In [2]:
# List of countries
c_list = ['DE', 
          'FR', 
          'IT', 
          'ES', 
          'PT', 
          'GR', 
          'NL', 
          'BE', 
          'AT', 
          'IE']

param = [('dataflow', 'IRS'),
         ('freq', 'M'),
         ('countries', '+'.join(c_list)),
         ('series', 'L'),
         ('trans type', 'L40'),
         ('maturity cat', 'CI'),
         ('counterpart', '0000'),
         ('currency', 'EUR'),
         ('business coverage', 'N'),
         ('interest rate type', 'Z'),
         ('start', '?startPeriod=2012-01-01')]

param_joined = '.'.join(value for key, value in param[1:-1])

series_key = '{}/{}{}'.format(param[0][1],
                              param_joined,
                              param[-1][1])

## Request data

In [3]:
url = 'https://sdw-wsrest.ecb.europa.eu/service/data/'
# headers used as content negotiation to return data in json format
headers = {'Accept':'application/json'}
r = requests.get('{}{}'.format(url, series_key), headers=headers).json()

## Pandas DataFrame

In [4]:
date_list = r['structure']['dimensions']['observation'][0]['values']
dates = [date['start'][:10] for date in date_list]
    
areas = [v['name'] for v in r['structure']['dimensions']['series'][1]['values']]

df = pd.DataFrame()
for i, area in enumerate(areas):
    s_key = '0:{}:0:0:0:0:0:0:0'.format(i)
    s_list = r['dataSets'][0]['series'][s_key]['observations']
    df[area] = pd.Series([s_list[val][0] for val in sorted(s_list, key=int)])
df.index = dates
df.tail()

,Austria,Belgium,Germany,Spain,France,Greece,Ireland,Italy,Netherlands,Portugal
2017-12-01,0.5027,0.53,0.30,1.443,0.67,4.44,0.54,1.795,0.452,1.828
2018-01-01,0.6692,0.70,0.47,1.473,0.86,3.79,0.91,1.984,0.611,1.846
2018-02-01,0.8357,0.97,0.66,1.509,0.98,4.14,1.13,2.081,0.761,2.032
2018-03-01,0.8134,0.87,0.53,1.334,0.84,4.27,1.01,1.965,0.626,1.790
2018-04-01,0.7625,0.81,0.48,1.206,0.78,4.04,0.94,1.773,0.690,1.662
